In [1]:
import os
#Set current location to the location of the script
os.chdir(os.path.dirname("/dust3r/playground.ipynb"))
print(os.getcwd())

/dust3r


In [ ]:
#Display imgs
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

def display_img(img):
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [2]:
from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda'
batch_size = 1
schedule = 'cosine'
lr = 0.01
niter = 300

model_name = "checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth"
# you can put the path to a local checkpoint in model_name if needed
model = AsymmetricCroCo3DStereo.from_pretrained(model_name).to(device)
# load_images can take a list of images or a directory
images = load_images(['croco/assets/Chateau1.png', 'croco/assets/Chateau2.png'], size=512)

... loading model from checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth
instantiating : AsymmetricCroCo3DStereo(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100', patch_embed_cls='PatchEmbedDust3R', img_size=(512, 512), head_type='dpt', output_mode='pts3d', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), landscape_only=False)
<All keys matched successfully>
>> Loading a list of 2 images
 - adding croco/assets/Chateau1.png with resolution 224x224 --> 512x384
 - adding croco/assets/Chateau2.png with resolution 224x224 --> 512x384
 (Found 2 images)


In [4]:
pairs = make_pairs(images, scene_graph='complete', prefilter=None, symmetrize=True)
output = inference(pairs, model, device, batch_size=batch_size)

>> Inference with model on 2 image pairs


100%|██████████| 2/2 [00:02<00:00,  1.34s/it]


In [5]:
view1, pred1 = output['view1'], output['pred1']
view2, pred2 = output['view2'], output['pred2']

In [9]:
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
loss = scene.compute_global_alignment(init="mst", niter=niter, schedule=schedule, lr=lr)

 init edge (1*,0*) score=31.983793258666992
 init loss = 0.004671978764235973
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals']


100%|██████████| 300/300 [00:17<00:00, 16.94it/s, lr=1.27413e-06 loss=0.00293712]


In [10]:
imgs = scene.imgs
focals = scene.get_focals()
poses = scene.get_im_poses()
pts3d = scene.get_pts3d()
confidence_masks = scene.get_masks()

In [20]:
print(pts3d)

[tensor([[[ 0.0387, -0.0226,  0.1448],
         [ 0.0614, -0.0179,  0.1242],
         [ 0.0617, -0.0179,  0.1240],
         ...,
         [ 0.0638, -0.0294,  0.2559],
         [ 0.0638, -0.0294,  0.2566],
         [ 0.0627, -0.0298,  0.2597]],

        [[ 0.0512, -0.0199,  0.1334],
         [ 0.0664, -0.0168,  0.1196],
         [ 0.0655, -0.0170,  0.1206],
         ...,
         [ 0.0616, -0.0299,  0.2613],
         [ 0.0626, -0.0296,  0.2595],
         [ 0.0617, -0.0300,  0.2621]],

        [[ 0.0584, -0.0183,  0.1267],
         [ 0.0673, -0.0165,  0.1188],
         [ 0.0670, -0.0166,  0.1192],
         ...,
         [ 0.0633, -0.0291,  0.2573],
         [ 0.0631, -0.0293,  0.2582],
         [ 0.0622, -0.0296,  0.2608]],

        ...,

        [[ 0.0512,  0.0240,  0.1300],
         [ 0.0516,  0.0239,  0.1298],
         [ 0.0518,  0.0239,  0.1297],
         ...,
         [ 0.1025,  0.0252,  0.1554],
         [ 0.1024,  0.0253,  0.1559],
         [ 0.1024,  0.0254,  0.1563]],

        [

In [23]:
for img in pts3d:
    img = img.detach().cpu().numpy()
    #Convert from bool to int
    for i in range(len(img)):
        for j in range(len(img[i])):
            img[i][j] = int(img[i][j])
    display_img(img)

(384, 512, 3)


TypeError: only length-1 arrays can be converted to Python scalars